In [1]:
import os
import torch
import torch.nn as nn
import numpy as np

In [2]:
from torchvision import models

In [3]:
from models.seresnet.FastPose import createModel

In [4]:
from src.opt import opt

In [5]:
weights = "test_weight/ceiling_0911_s/to17kps_s5E-7_acc/to17kps_s5E-7_best_acc.pkl"

In [6]:
model =  createModel(cfg=None).cpu()

In [7]:
model.load_state_dict(torch.load(weights, map_location="cpu"))

<All keys matched successfully>

In [8]:
def obtain_prune_idx(path):
    lines = []
    with open(path, 'r') as f:
        file = f.readlines()
        for line in file:
            lines.append(line)
            
    idx = 0
    prune_idx = []
    for line in lines:
        if "):" in line:
            idx  += 1
        if "BatchNorm2d" in line:
            #print(idx, line)
            prune_idx.append(idx)
    
    prune_idx = prune_idx[1:] # 去除第一个bn1层
    return prune_idx

model_name = "./model.txt"
print(model, file=open(model_name, 'w'))
prune_idx = obtain_prune_idx(model_name)

In [9]:
len(prune_idx)

105

In [10]:
def sort_bn(model, prune_idx):
    size_list = [m.weight.data.shape[0] for idx, m in enumerate(model.modules()) if idx in prune_idx]
    # bn_layer = [m for m in model.modules() if isinstance(m, nn.BatchNorm2d)]
    bn_prune_layers = [m for idx, m in enumerate(model.modules()) if idx in prune_idx]
    bn_weights = torch.zeros(sum(size_list))

    index = 0
    for module, size in zip(bn_prune_layers, size_list):
        bn_weights[index:(index + size)] = module.weight.data.abs().clone()
        index += size
    sorted_bn = torch.sort(bn_weights)[0]
    
    return sorted_bn

sorted_bn = sort_bn(model, prune_idx)

In [11]:
sorted_bn

tensor([1.5832e-11, 1.5832e-11, 4.4936e-11,  ..., 6.7178e-01, 6.9209e-01,
        7.3103e-01])

In [12]:
def obtain_bn_mask(bn_module, thre):
    if device != "cpu":
        thre = thre.cuda()
    mask = bn_module.weight.data.abs().ge(thre).float()

    return mask

def obtain_filters_mask(model, prune_idx, thre):
    pruned = 0
    bn_count = 0
    total = 0
    num_filters = []
    pruned_filters = []
    filters_mask = []
    pruned_maskers = []
    
    for idx, module in enumerate(model.modules()):
        if isinstance(module, nn.BatchNorm2d):
            if idx in prune_idx:
                mask = obtain_bn_mask(module, thre).cpu().numpy()
                remain = int(mask.sum())
                pruned = pruned + mask.shape[0] - remain

                if remain == 0: # 保证至少有一个channel
                    # print("Channels would be all pruned!")
                    # raise Exception
                    max_value = module.weight.data.abs().max()
                    mask = obtain_bn_mask(module, max_value).cpu().numpy()
                    remain = int(mask.sum())
                    pruned = pruned + mask.shape[0] - remain
                    bn_count += 1
                print(f'layer index: {idx:>3d} \t total channel: {mask.shape[0]:>4d} \t '
                      f'remaining channel: {remain:>4d}')
                
                pruned_filters.append(remain)
                pruned_maskers.append(mask.copy())
            else:
                mask = np.ones(module.weight.data.shape)
                remain = mask.shape[0]
            
            total += mask.shape[0]
            num_filters.append(remain)
            filters_mask.append(mask.copy())
    
    prune_ratio = pruned / total
    print(f'Prune channels: {pruned}\tPrune ratio: {prune_ratio:.3f}')

    return pruned_filters, pruned_maskers

In [13]:
percent = 0.5

In [14]:
def obtain_bn_threshold(model, sorted_bn, percentage):
    thre_index = int(len(sorted_bn) * percentage)
    thre = sorted_bn[thre_index]
    
    return thre

threshold = obtain_bn_threshold(model, sorted_bn, percent)
print(threshold)

tensor(0.0002)


In [15]:
device = "cpu"

In [16]:
pruned_filters, pruned_maskers = obtain_filters_mask(model, prune_idx, threshold)

layer index:   9 	 total channel:   64 	 remaining channel:   46
layer index:  11 	 total channel:   64 	 remaining channel:   41
layer index:  13 	 total channel:  256 	 remaining channel:  154
layer index:  23 	 total channel:  256 	 remaining channel:  167
layer index:  26 	 total channel:   64 	 remaining channel:   20
layer index:  28 	 total channel:   64 	 remaining channel:   25
layer index:  30 	 total channel:  256 	 remaining channel:  159
layer index:  33 	 total channel:   64 	 remaining channel:   48
layer index:  35 	 total channel:   64 	 remaining channel:   59
layer index:  37 	 total channel:  256 	 remaining channel:  182
layer index:  41 	 total channel:  128 	 remaining channel:  106
layer index:  43 	 total channel:  128 	 remaining channel:  124
layer index:  45 	 total channel:  512 	 remaining channel:  328
layer index:  55 	 total channel:  512 	 remaining channel:  333
layer index:  58 	 total channel:  128 	 remaining channel:   62
layer index:  60 	 total 

In [17]:
print(pruned_filters, file=open("ceiling.txt", "w"))

In [18]:
new_model = createModel(cfg="ceiling.txt").cpu()

In [19]:
new_model

FastPose(
  (preact): SeResnet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): SeBottleneck(
        (conv1): Conv2d(64, 46, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(46, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(46, 41, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(41, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(41, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (se): SELayer(
          (avg_pool): AdaptiveAvgPool2d(output_size